# LINEAR_ASSIGN

## Overview
The LINEAR_ASSIGN function solves the classic assignment problem (Hungarian algorithm) using the `scipy.optimize.linear_sum_assignment` method. It finds the optimal assignment that minimizes the total cost for a given cost matrix.

## Usage
To use the `LINEAR_ASSIGN` function in Excel, enter it as a formula in a cell, specifying your cost matrix as a 2D list or Excel range:

```excel
=LINEAR_ASSIGN(cost_matrix)
```

## Parameters
| Parameter    | Type     | Required | Description                                 |
|--------------|----------|----------|---------------------------------------------|
| cost_matrix  | 2D list  | Yes      | The cost matrix (m x n) for the assignment. |

## Return Value
| Return Value | Type     | Description                                                      |
|--------------|----------|------------------------------------------------------------------|
| Assignment   | 2D list  | A 2D list of [row, col] pairs representing the optimal assignment.|

## Limitations
- The input must be a 2D list (matrix) of numbers.
- The function returns the assignment as a list of [row, col] index pairs.
- If the input is invalid, an error message string is returned.
- Only square or rectangular matrices are supported.
- Requires `scipy` (available in Pyodide).

## Benefits
- Solves assignment problems in operations research, scheduling, and logistics.
- Automates optimal resource allocation directly in Excel.
- Fast and reliable for small to medium-sized matrices.

## Examples

### Task Assignment
Assign 3 workers to 3 tasks to minimize total cost.

**Input:**
|   |   |   |
|---|---|---|
| 4 | 1 | 3 |
| 2 | 0 | 5 |
| 3 | 2 | 2 |

**Sample Call:**
```excel
=LINEAR_ASSIGN(A1:C3)
```
**Sample Output:**
```
[[0, 1], [1, 0], [2, 2]]
```

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def linear_assign(cost_matrix):
    """
    Solves the linear assignment problem (Hungarian algorithm) for a given cost matrix.

    Args:
        cost_matrix (list): 2D list representing the cost matrix.

    Returns:
        list: 2D list of [row, col] assignments, or error message string.
    """
    if linear_sum_assignment is None:
        return "scipy is required but not available."
    try:
        arr = np.array(cost_matrix)
        if arr.ndim != 2:
            return "Input must be a 2D list (matrix)."
        row_ind, col_ind = linear_sum_assignment(arr)
        assignment = [[int(r), int(c)] for r, c in zip(row_ind, col_ind)]
        return assignment
    except Exception as e:
        return str(e)

In [ ]:
# Unit Tests
import ipytest
ipytest.autoconfig()

def test_demo_basic_assignment():
    cost_matrix = [
        [4, 1, 3],
        [2, 0, 5],
        [3, 2, 2]
    ]
    result = linear_assign(cost_matrix)
    assert isinstance(result, list)
    assert all(isinstance(pair, list) and len(pair) == 2 for pair in result)
    assert len(result) == 3
    assert sorted(result) == sorted([[0, 1], [1, 0], [2, 2]])

def test_rectangular_matrix():
    cost_matrix = [
        [10, 19, 8, 15],
        [10, 18, 7, 17],
        [13, 16, 9, 14]
    ]
    result = linear_assign(cost_matrix)
    assert isinstance(result, list)
    assert all(isinstance(pair, list) and len(pair) == 2 for pair in result)
    assert len(result) == 3

def test_invalid_input_not_2d():
    cost_matrix = [1, 2, 3]
    result = linear_assign(cost_matrix)
    assert isinstance(result, str) and len(result) > 0

def test_empty_matrix():
    cost_matrix = []
    result = linear_assign(cost_matrix)
    assert isinstance(result, str) and len(result) > 0

def test_negative_costs():
    cost_matrix = [
        [-1, -2, -3],
        [-2, -4, -6],
        [-3, -6, -9]
    ]
    result = linear_assign(cost_matrix)
    assert isinstance(result, list)
    assert all(isinstance(pair, list) and len(pair) == 2 for pair in result)
    assert len(result) == 3

ipytest.run()

In [3]:
# Interactive Demo
import gradio as gr

examples = [
    [
        [
            [4, 1, 3],
            [2, 0, 5],
            [3, 2, 2]
        ]
    ],
    [
        [
            [10, 19, 8, 15],
            [10, 18, 7, 17],
            [13, 16, 9, 14]
        ]
    ]
]

demo = gr.Interface(
    fn=linear_assign,
    inputs=gr.Dataframe(
        headers=None,
        label="Cost Matrix",
        row_count=3,
        col_count=3,
        type="array",
        value=[
            [4, 1, 3],
            [2, 0, 5],
            [3, 2, 2]
        ]
    ),
    outputs=gr.Dataframe(headers=["Row", "Col"], label="Assignment"),
    examples=examples,
    description="Solve the linear assignment problem (Hungarian algorithm) for a given cost matrix.",
    flagging_mode="never",
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
